In [9]:
from TradingAgent import PairsTradingAgent
import numpy as np
import pandas as pd
from utils import compute_spread, zscore_normalization
from copy import deepcopy as dc
from tqdm import tqdm
import utils

In [3]:
raw_data = pd.read_csv("../data/raw_data.csv", index_col=0)
raw_data.index = pd.to_datetime(raw_data.index)
raw_data.head()

,A,AAL,AAPL,ABBV,ABT,ACGL,ACN,ADBE,ADI,ADM,...,WTW,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2016-01-04,40.689999,40.910000,26.337500,57.610001,42.930000,22.950001,101.830002,91.970001,54.439999,35.840000,...,124.980133,29.83,68.769997,35.700001,77.459999,36.080002,51.912292,98.844658,66.489998,47.270000
2016-01-05,40.549999,40.520000,25.677500,57.369999,42.919998,23.033333,102.360001,92.339996,54.040001,36.240002,...,125.839996,29.90,70.070000,36.060001,78.120003,36.070000,51.782890,100.902916,64.820000,48.009998
2016-01-06,40.730000,41.230000,25.174999,57.380001,42.560001,23.070000,102.160004,91.019997,51.740002,35.360001,...,125.839996,29.24,66.440002,36.439999,77.470001,35.619999,51.416248,101.339806,62.230000,48.020000
2016-01-07,39.000000,40.450001,24.112499,57.209999,41.540001,23.046667,99.160004,89.110001,50.419998,34.520000,...,114.949997,28.16,60.189999,36.580002,76.230003,34.700001,49.662113,99.009712,59.410000,46.560001
2016-01-08,38.590000,40.369999,24.240000,55.650002,40.669998,22.806667,98.199997,87.849998,49.980000,34.389999,...,116.620003,27.90,57.740002,36.180000,74.690002,34.369999,48.986340,98.592232,59.250000,45.880001


In [8]:
def grid_search(trading_agent: PairsTradingAgent, interval: float, 
                if_fixed_capital:bool, trading_capital=1.0, trading_capital_ratio=0.05):
    best_return = -np.inf
    best_entry_threshold = 0
    for entry_threshold in np.arange(0.8, 2.0, interval):   
        backtest_return_series = trading_agent.trading_simulation(entry=entry_threshold, exit=0.5, if_fixed_capital=if_fixed_capital,
                                                                  trading_capital=trading_capital, trading_capital_ratio=trading_capital_ratio)
        backtest_return = backtest_return_series.iloc[-1].values[0]
        if backtest_return > best_return:
            best_return = backtest_return
            best_entry_threshold = entry_threshold
    
    return best_entry_threshold, best_return

In [2]:
trading_pairs = pd.read_csv("../cluster_find_trading_pairs/trading_pairs.csv", index_col=0)
# trading_pairs.insert(loc=len(trading_pairs.columns), column="best_threshold", value=None)
# trading_pairs.insert(loc=len(trading_pairs.columns), column="best_return", value=None)
# trading_pairs.head()

In [10]:
for ind in tqdm(trading_pairs.index):
    leg1 = trading_pairs.loc[ind, "leg1"]
    leg2 = trading_pairs.loc[ind, "leg2"]

    spread = compute_spread(raw_data, leg1, leg2)
    spread = zscore_normalization(spread)
    
    trading_agent = PairsTradingAgent(raw_data.loc[:, leg1], raw_data.loc[:, leg2], spread, initial_capital=1.0)
    best_entry_threshold, best_return = grid_search(trading_agent, interval=0.05, if_fixed_capital=True, trading_capital=1.0)
    
    trading_pairs.loc[ind, "best_threshold"] = best_entry_threshold
    trading_pairs.loc[ind, "best_return"] = best_return
    # trading_pairs.to_csv("./trading_pairs_potential_fixed_capital.csv")
    # break

100%|██████████| 2069/2069 [4:05:27<00:00,  7.12s/it]  


In [4]:
trading_pairs = pd.read_csv("./trading_pairs_potential_fixed_capital.csv", index_col=0)
trading_pairs.head()

,leg1,leg2,coint_t,coint_pvalue,hedge_ratio,adf_pvalue,hurst_exponent,half_life,zero_cross,best_threshold,best_return
0,A,AVY,-3.992017,0.007372,1.245725,0.001497,0.397680,37.003035,117,0.80,2.695672
1,A,DGX,-4.205360,0.003580,0.561827,0.000161,0.422547,36.591205,96,0.80,3.297833
2,A,MS,-3.843708,0.011842,0.597435,0.002614,0.407522,50.032469,78,0.80,3.320636
3,A,MTD,-3.814590,0.012961,10.115935,0.001257,0.418362,40.976883,105,0.80,2.307756
4,A,NDAQ,-4.611753,0.000798,0.395134,0.000121,0.373605,29.043458,90,1.15,3.236758


In [5]:
sorted_trading_pairs = trading_pairs.sort_values("best_return", ascending=False)
sorted_trading_pairs.head(n=10)

,leg1,leg2,coint_t,coint_pvalue,hedge_ratio,adf_pvalue,hurst_exponent,half_life,zero_cross,best_threshold,best_return
901,DUK,ENPH,-3.922743,0.009225,7.188884,0.006725,0.409722,52.154031,111,0.95,10.517144
849,DLTR,ENPH,-3.673073,0.019838,2.724662,0.018532,0.407459,70.402656,59,0.80,10.195229
742,CZR,ETSY,-4.411852,0.001703,2.288153,0.006295,0.440650,57.508409,60,1.60,8.490281
744,CZR,NKE,-4.245217,0.003112,1.159681,0.013336,0.407289,65.688975,38,1.20,8.342575
319,APTV,ETSY,-4.130476,0.004638,1.955878,0.001353,0.376077,46.333395,121,0.90,7.031082
741,CZR,EPAM,-4.372294,0.001970,5.230379,0.014134,0.456380,74.706082,40,1.55,6.760948
1287,HOLX,TSLA,-3.800707,0.013526,6.041251,0.013859,0.449642,64.927930,87,0.85,6.687967
1045,ENPH,JNJ,-4.055910,0.005967,0.186662,0.000147,0.405688,39.557506,83,0.90,6.653791
159,ALGN,CZR,-5.141951,0.000090,0.147969,0.000114,0.373032,34.104557,92,1.70,6.388090
970,DXCM,SPGI,-3.792384,0.013876,2.452707,0.002625,0.413481,48.201554,55,0.85,6.118240


In [6]:
top_trading_pairs = sorted_trading_pairs[:300]
top_trading_pairs.to_csv("./top_trading_pairs.csv")

In [7]:
top_trading_paris = pd.read_csv("./top_trading_pairs.csv", index_col=0)
top_trading_paris.index = range(len(top_trading_paris))
top_trading_paris.head(n=10)

,leg1,leg2,coint_t,coint_pvalue,hedge_ratio,adf_pvalue,hurst_exponent,half_life,zero_cross,best_threshold,best_return
0,DUK,ENPH,-3.922743,0.009225,7.188884,0.006725,0.409722,52.154031,111,0.95,10.517144
1,DLTR,ENPH,-3.673073,0.019838,2.724662,0.018532,0.407459,70.402656,59,0.80,10.195229
2,CZR,ETSY,-4.411852,0.001703,2.288153,0.006295,0.440650,57.508409,60,1.60,8.490281
3,CZR,NKE,-4.245217,0.003112,1.159681,0.013336,0.407289,65.688975,38,1.20,8.342575
4,APTV,ETSY,-4.130476,0.004638,1.955878,0.001353,0.376077,46.333395,121,0.90,7.031082
5,CZR,EPAM,-4.372294,0.001970,5.230379,0.014134,0.456380,74.706082,40,1.55,6.760948
6,HOLX,TSLA,-3.800707,0.013526,6.041251,0.013859,0.449642,64.927930,87,0.85,6.687967
7,ENPH,JNJ,-4.055910,0.005967,0.186662,0.000147,0.405688,39.557506,83,0.90,6.653791
8,ALGN,CZR,-5.141951,0.000090,0.147969,0.000114,0.373032,34.104557,92,1.70,6.388090
9,DXCM,SPGI,-3.792384,0.013876,2.452707,0.002625,0.413481,48.201554,55,0.85,6.118240


In [10]:
spread_list = []
for ind in top_trading_paris.index:
    leg1 = top_trading_paris.loc[ind, "leg1"]
    leg2 = top_trading_paris.loc[ind, "leg2"]
    spread = utils.compute_spread(raw_data, leg1, leg2)
    spread = utils.zscore_normalization(spread)
    spread = pd.DataFrame(spread, columns=[f"{leg1}-{leg2}"])
    spread_list.append(spread)

spread_df = pd.concat(spread_list, axis=1)
spread_df.index = raw_data.index
spread_df.index = pd.to_datetime(spread_df.index)
spread_df.to_csv("./spread.csv")

In [12]:
spread_df.head(n=10)

,DUK-ENPH,DLTR-ENPH,CZR-ETSY,CZR-NKE,APTV-ETSY,CZR-EPAM,HOLX-TSLA,ENPH-JNJ,ALGN-CZR,DXCM-SPGI,...,DE-MNST,COR-PWR,CMI-NXPI,ABT-ADSK,MKC-PCG,TSLA-UPS,GEN-STT,ACN-BX,AMT-PAYC,CHRW-RF
Date,,,,,,,,,,,,,,,,,,,,,
2016-01-04,0.950779,-0.062335,0.338061,0.493245,-0.960299,0.321404,0.196262,-2.826972,-0.196873,-1.877955,...,0.238320,-2.820343,1.882292,-0.726033,-0.766538,-0.943261,-0.562159,1.502222,0.329124,-0.319540
2016-01-05,0.885127,-0.123211,0.307581,0.512641,-0.816526,0.286389,0.185758,-2.784137,-0.148704,-1.909758,...,0.302001,-3.034343,1.823101,-0.721826,-0.750898,-0.881910,-0.560646,1.388772,0.233845,-0.471109
2016-01-06,0.861985,-0.153599,0.316902,0.450443,-0.681268,0.275198,0.121562,-2.829710,-0.146069,-1.915541,...,0.331167,-2.872227,1.741861,-0.722092,-0.789169,-0.964875,-0.756963,1.304690,0.249806,-0.498458
2016-01-07,0.941003,-0.053351,0.348278,0.373244,-0.429414,0.279369,0.240014,-2.939196,-0.175887,-2.046444,...,0.299470,-2.572446,1.598162,-0.769696,-0.846792,-1.079206,-0.954208,1.343512,0.356205,-0.593807
2016-01-08,0.960660,-0.020638,0.337360,0.304837,-0.355493,0.254893,0.329921,-3.040803,-0.193072,-2.083635,...,0.286775,-2.528930,1.613722,-0.724199,-0.890212,-1.152031,-1.098528,1.291285,0.376419,-0.663981
2016-01-11,0.859431,-0.002680,0.304559,0.351749,-0.518825,0.251040,0.380823,-3.094058,-0.130040,-2.052476,...,0.354135,-2.141477,1.743407,-0.723002,-0.847603,-1.130213,-1.092732,1.150319,0.361082,-0.572927
2016-01-12,0.927274,-0.071114,0.310995,0.388238,-0.470708,0.262967,0.316884,-3.028111,-0.158264,-2.046374,...,0.399399,-2.151787,1.713069,-0.810482,-0.737848,-1.045535,-1.064524,1.125096,0.366042,-0.658672
2016-01-13,0.870317,0.085538,0.298433,0.308212,-0.206785,0.259380,0.442410,-3.141504,-0.106303,-2.055309,...,0.274270,-2.073696,1.621442,-0.857481,-0.799192,-1.188748,-1.223134,1.126971,0.414816,-0.772231
2016-01-14,0.784658,0.071830,0.313994,0.302364,-0.161901,0.304777,0.462897,-2.959254,-0.145780,-2.059299,...,0.212530,-2.137241,1.548924,-0.863226,-0.678593,-1.161293,-1.209308,1.136467,0.429175,-0.800019
